In [52]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from IPython.display import display, Image
from scipy import stats
from sklearn.decomposition import PCA
import statsmodels.api as sm


In [5]:
rnaseq = pd.read_csv('mTEC_RNAseq_results_all.csv')
tss=pd.read_pickle('df_all_new.pkl')


,Chr,Start,Stop,Samples,Sample_indices,Strand,Listing,Listing_subject,High,Low,...,Annotations_short,CpG%,GC%,Housekeeping,TRA,Aire_dep,Fezf2_dep,Aire_dep_TRA,Fezf2_dep_TRA,Other_TRA
0,chr1,778661,778688,"pt212_hi,pt87_lo","pt212_hi_1011,pt87_lo_1221",-,2,2,1,1,...,promoter-TSS,0.100000,0.677205,False,False,False,False,False,False,False
1,chr1,827506,827522,pt212_hi,pt212_hi_1012,-,1,1,1,0,...,promoter-TSS,0.128333,0.715474,False,False,False,False,False,False,False
2,chr1,827657,827703,"pt212_hi,pt214_lo,pt87_lo","pt212_hi_1,pt214_lo_1,pt87_lo_1",+,3,3,1,2,...,promoter-TSS,0.111667,0.680532,False,False,False,False,False,False,False
3,chr1,940498,940501,pt87_lo,pt87_lo_2,+,1,1,0,1,...,intron,0.100000,0.703827,False,False,False,False,False,False,False
4,chr1,945627,945644,pt212_lo,pt212_lo_1067,-,1,1,0,1,...,exon,0.025000,0.599002,True,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52028,chrY,19567833,19567843,pt212_hi,pt212_hi_17807,+,1,1,1,0,...,intron,0.060000,0.600666,False,False,False,False,False,False,False
52029,chrY,19706330,19706336,pt212_lo,pt212_lo_20141,-,1,1,0,1,...,exon,0.025000,0.565724,False,False,False,False,False,False,False
52030,chrY,19744721,19744728,"pt212_hi,pt212_lo","pt212_hi_17816,pt212_lo_20142",-,2,1,1,1,...,promoter-TSS,0.060000,0.470882,False,False,False,False,False,False,False
52031,chrY,20575712,20575803,"pt212_hi,pt212_lo,pt221_hi,pt221_lo,pt226_hi,p...","pt212_hi_17808,pt212_hi_17809,pt212_hi_17810,p...",+,7,4,3,4,...,promoter-TSS,0.043333,0.489185,False,False,False,False,False,False,False


In [39]:
tss_protein = tss[tss.Gene_Type=="protein-coding"]

expression_mask=rnaseq.loc[:,rnaseq.columns.str.startswith('pt')] > 1
expressed = rnaseq[expression_mask.any(axis=1)]
expressed_protein = expressed[expressed.type=="protein-coding"]

(13259,)

In [49]:
tss_rnaseq_protein_intersect = np.intersect1d(expressed_protein.ens_gene.dropna().values, tss_protein.Nearest_Ensembl.dropna().values)
tss_rnaseq_protein_intersect.shape[0]/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]

0.9612338788747266

In [50]:
tss_rnaseq_intersect = np.intersect1d(expressed.ens_gene.dropna().values, tss.Nearest_Ensembl.dropna().values).shape
tss_rnaseq_intersect[0]/tss.Nearest_Ensembl.dropna().unique().shape[0]

0.8513875285667646

In [62]:
both=tss_rnaseq_protein_intersect.shape[0] 
tss_only = tss_protein.Nearest_Ensembl.dropna().unique().shape[0] - both
rnaseq_only = expressed_protein.ens_gene.dropna().unique().shape[0] - both

c=venn2(subsets = (tss_only, rnaseq_only, both),alpha=0.7,set_colors=('blue','gray'),set_labels = ('5P-seq', 'RNAseq'))
plt.savefig('figureS5.pdf', bbox_inches="tight")
plt.close()

print(f'Total TSRs in protein-coding genes (multiple TSS per gene possible): {tss_protein.Nearest_Ensembl.dropna().unique().shape[0]}')
print(f'Protein-coding genes expressed (multiple transcripts per gene possible): {expressed_protein.ens_gene.dropna().unique().shape[0]}')
print(f'Genes for which both TRS and RNAseq evidence for expression:  {both} ({round(both/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]*100,1)}%)')
print(f'Fraction of TRS: TRS only and shared with RNAseq evidence for expression:  {round(tss_only/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]*100,1)}% and {round(both/tss_protein.Nearest_Ensembl.dropna().unique().shape[0]*100,1)}%')
print(f'Fraction of expressed genes: expressed only and shared with TRS evidence:  {round(rnaseq_only/expressed_protein.ens_gene.dropna().unique().shape[0]*100,1)}% and {round(both/expressed_protein.ens_gene.dropna().unique().shape[0]*100,1)}%')


Total TSRs in protein-coding genes (multiple TSS per gene possible): 13259
Protein-coding genes expressed (multiple transcripts per gene possible): 15770
Genes for which both TRS and RNAseq evidence for expression:  12745 (96.1%)
Fraction of TRS: TRS only and shared with RNAseq evidence for expression:  3.9% and 96.1%
Fraction of expressed genes: expressed only and shared with TRS evidence:  19.2% and 80.8%
